# Download Data

In [2]:
import requests
import pandas as pd
from io import BytesIO

def download_csv(csv_url):
    response = requests.get(csv_url)
    response.raise_for_status()

    # Qui puoi modificare 'sep' o aggiungere altri parametri se necessario
    try:
        df = pd.read_csv(BytesIO(response.content), sep=',')
    except pd.errors.ParserError:
        df = pd.read_csv(BytesIO(response.content), sep=';', error_bad_lines=False)

    return df

In [10]:
recipes_df = download_csv("https://github.com/GiovTemp/SustainaMeal_Case_Study/raw/main/data/valid_recipes_dataset.csv")

In [11]:
recipes_df

,title,sustainability_score,sustainability_label,Unnamed: 0,recipe_id,description,author_id,duration,directions,ingredients,...,sugars [g],protein [g],direction_size,ingredients_sizes,who_score,fsa_score,nutri_score,normalization_comment,ingredient_food_kg_urls,ingredient_food_kg_names
0,Jack O Lantern 'o Fragrance,0.002610,0,495656,466123,I suppose that this isn't really a recipe as m...,169430,30.0,['When you cut the lid out of the pumpkin do ...,"['pumpkin liking', 'candle pumpkin', 'pumpkin ...",...,0.3,0.6,7,4,0.302381,0.250,0.25,NaN,NaN,NaN
1,Boiled Radishes,0.002711,0,294268,179840,These taste a lot like new potatoes. The boil...,254469,22.0,"['Trim ends and bad spots off Radishes.', 'Boi...","['radishes size bag bunch', 'water radishes']",...,2.4,0.8,2,2,0.293040,1.000,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['applesauce', 'garlic clove', 'ginger', 'mola..."
2,Horseradish Applesauce,0.002807,0,364381,174389,"I'm not sure this even qualifies as a recipe, ...",226918,22.0,"['Blend applesauce and horseradish. ', 'Refrig...","['applesauce', 'horseradish']",...,0.6,0.3,2,2,0.313757,0.875,0.50,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['all - purpose flour', 'butter', 'butternut s..."
3,Survival Necklace,0.002842,0,43445,95937,Fun for kids to make and a great way to keep t...,25792,10.0,['Other supplies: 18 inches string or yarn or ...,['Fruit Loops cereal cup apple jacks kind hole...,...,1.2,8.8,7,2,0.319215,0.750,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Old Bay Seasoning', 'Worcestershire sauce', ..."
4,Apple Cider Reduction,0.002842,0,59722,521756,I saw a post about this on Facebook and gave i...,171084,155.0,"['In 6-8 quart stock pan, measure out 2 cups o...","['gallon apple cider fresh', 'gallon apple cid...",...,0.0,0.0,10,2,0.214286,0.750,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['bean sprouts', 'chicken thighs', 'fresh ging..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100865,Curried Lamb on Rice,1.030416,2,103145,373244,"With modification this recipe is from ""The Dia...",133174,65.0,"['Over medium heat, melt the butter in a large...","['lamb', 'medium onion', 'cloves', 'butter', '...",...,3.4,28.7,5,13,0.223549,0.625,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Dijon mustard', 'Polish sausage', 'apricot p..."
100866,Rich Lamb Curry,1.040688,2,442694,268247,The recipe isn't as time consuming as it looks...,491437,110.0,"['Heat 1 tbsp olive oil in dutch oven. ', 'Co...","['oil', 'lamb', 'flour', 'salt pepper', 'onion...",...,30.4,36.1,9,21,0.153328,0.000,0.00,NaN,NaN,NaN
100867,"Middle Eastern Slow-Cooked Stew With Lamb, Chi...",1.049306,2,359960,144850,From Cooking Light. Per 3/4 c. serving: 310 ca...,37779,112.0,['Let the oil get heating in a large pot over ...,"['oil', 'lamb', 'onions rings', 'water', 'clov...",...,20.1,20.8,12,19,0.205566,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['anchovies', 'celery ribs', 'diced tomatoes',..."
100868,Five Meat Chili Con Carne With Beans,1.052551,2,37637,13568,yum,20571,210.0,"['In a large pot brown meat in stages, about a...","['beef', 'pork', 'lamb', 'sausage', 'sausage',...",...,6.6,14.5,7,15,0.197568,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"[""French\\'s French fried onions"", 'crabmeat',..."


# Utils

In [6]:
!pip install git+https://github.com/GiovTemp/SustainaMeal.git

  Cloning https://github.com/GiovTemp/SustainaMeal.git to /tmp/pip-req-build-_5y9lusn
  Running command git clone --filter=blob:none --quiet https://github.com/GiovTemp/SustainaMeal.git /tmp/pip-req-build-_5y9lusn
  Resolved https://github.com/GiovTemp/SustainaMeal.git to commit f17c1ac20a93180af383cd06ff9771797f7e5c46
  Preparing metadata (setup.py) ... done


# SustainaMeal

Initializes the system by loading the data and preparing the embeddings.

In [12]:
from sustainameal import SustainaMeal

sm = SustainaMeal(
    recipes_df=recipes_df,
    nutrients=['calories [cal]', 'totalFat [g]', 'saturatedFat [g]', 'cholesterol [mg]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]'],
    transformer_name='davanstrien/autotrain-recipes-2451975973'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Processing Titles embeddings: 100%|██████████| 100870/100870 [16:46<00:00, 100.25batch/s]


## Find Similar Recipes by Title

Debug function which return the list of recipes titles similar to the given input text (STEP 1).

        
        """
        Parameters

        :param input_text: The input text to find similar recipes for.
        :param k: Number of similar recipes to return.
        
        :return: A list of tuples with similar recipes and their similarity scores.
        """

In [13]:
similar_by_title = sm.get_similar_by_title('Creamy Lemon Asparagus Risotto', 10)

Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 83.88batch/s]


In [14]:
df = pd.DataFrame(similar_by_title, columns=['Title', 'Similarity'])
df

,Title,Similarity
0,"(171027, Creamy Lemon Asparagus Risotto)",1.000000
1,"(153003, Creamy Spinach Risotto)",0.959703
2,"(237255, Creamy Lemon Fettuccine)",0.957575
3,"(167231, Creamy Apple Raisin Oatmeal)",0.957208
4,"(394589, Creamy Elegant Rice Casserole)",0.952827
5,"(218448, Creamy Blueberry Gelatin Salad)",0.951957
6,"(152709, Creamy Asparagus Meatloaf)",0.948904
7,"(249025, Creamy Cornbread Casserole)",0.948449
8,"(343589, Creamy Spinach Pasta Casserole)",0.946452
9,"(204569, Creamy Italian Pasta Salad)",0.946278


## Find Similar Recipes by tags and Title

Finds recipes similar (nutrients similarity) to the given input text. (STEP 1-2).
        
        """
        Parameters

        :param input_text: The input text to find similar recipes for.
        :param k: Number of similar recipes to return.
        :param acceptable_tags: List of tags considered acceptable for filtering recipes.
        :param match_all_tags: Matching strategy
        
        :return: A list of tuples with similar recipes and their similarity scores.
        """

In [15]:
sr = sm.find_similar_recipes('Creamy Lemon Asparagus Risotto', 10,
                        acceptable_tags=['appetizers', 'main-dish', 'side-dishes', 'fruits', 'desserts',
                                          'breakfast', 'pasta-rice-and-grains', 'beverages', 'drinks'],
                        match_all_tags=True)

Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 78.70batch/s]


Number of filtered recipes: 823


In [16]:
df = pd.DataFrame(sr)
df

,recipe_id,title,similarity
0,263276,"Rice Cooker "" Risotto"" With Carrots and Onions",0.998886
1,62907,Super Easy Pasta Salad,0.996178
2,17270,Quick and Easy Saffron Rice,0.995596
3,162565,Rice With Almonds & Raisins,0.995392
4,175027,Rice Cooker Rice Pilaf,0.995289
5,365442,Orange-Cinnamon Rice Pilaf,0.995237
6,334876,Italian Inspired Microwave Rice,0.994956
7,503443,Vietnamese Lemon Rice,0.994284
8,112225,Arroz Central Cafe (Central Cafe Rice),0.993926
9,272316,Perfect Risotto for Wolfgang Puck Pressure Cooker,0.993820


## Order results by healthiness score

  Order the recipes obtained previously.
        
        """

        :param (optional) nearest_recipes: Dataframe to order, if none the dataframe computed by find_similar_recipes will be used
        :param score (default:'who_score') : The column name used as the primary sorting criterion.

        :return: A DataFrame of recipes ordered by the specified score.

        """

In [17]:
sm.order_recipe_by_healthiness()

,title,who_score
40212,Arroz Central Cafe (Central Cafe Rice),0.293147
23549,Rice Cooker Rice Pilaf,0.248747
12804,Vietnamese Lemon Rice,0.237803
57340,Quick and Easy Saffron Rice,0.234049
5741,Super Easy Pasta Salad,0.230867
35258,Rice With Almonds & Raisins,0.220635
71749,Perfect Risotto for Wolfgang Puck Pressure Cooker,0.219227
76855,Italian Inspired Microwave Rice,0.218376
31929,Orange-Cinnamon Rice Pilaf,0.217653
32232,"Rice Cooker "" Risotto"" With Carrots and Onions",0.215539


## Order results by sustainability score

  Order the recipes obtained previously.
        
        """

        :param (optional) nearest_recipes: Dataframe to order, if none the dataframe computed by find_similar_recipes will be used
        :param score (default:'sustainability_label') : The column name used as the primary sorting criterion.
        :param secondary_sort_field (default:'who_score'): The column name used as the secondary sorting criterion.

        :return: A Dataframe with recipes ordered by the given metric.

        """

In [18]:
sm.order_recipe_by_sustainability()

,title,sustainability_label,who_score
5741,Super Easy Pasta Salad,0,0.230867
12804,Vietnamese Lemon Rice,0,0.237803
32232,"Rice Cooker "" Risotto"" With Carrots and Onions",1,0.215539
31929,Orange-Cinnamon Rice Pilaf,1,0.217653
76855,Italian Inspired Microwave Rice,1,0.218376
71749,Perfect Risotto for Wolfgang Puck Pressure Cooker,1,0.219227
35258,Rice With Almonds & Raisins,1,0.220635
57340,Quick and Easy Saffron Rice,1,0.234049
23549,Rice Cooker Rice Pilaf,1,0.248747
40212,Arroz Central Cafe (Central Cafe Rice),1,0.293147
